<a id="title_ID"></a>
# JWST Pipeline Validation Notebook: 
# AMI3, AMI3

<span style="color:red"> **Instruments Affected**</span>: NIRISS

### Table of Contents

<div style="text-align: left"> 
    
<br> [Introduction](#intro)
<br> [JWST CalWG Algorithm](#algorithm)
<br> [Defining Terms](#terms)
<br> [Test Description](#description)
<br> [Data Description](#data_descr)
<br> [Set up Temporary Directory](#tempdir)
<br> [Imports](#imports)
<br> [Loading the Data](#data_load)
<br> [Run the Pipeline](#pipeline)
<br> [Test Results](#testing)
<br> [About This Notebook](#about)  

</div>

<a id="intro"></a>
# Introduction

The notebook verifies that pipeline steps from `calwebb_detector1` through `calwebb_image2` and `calwebb_ami3` run without crashing. `calwebb_ami3` is run on various associations of target and calibrator pairs.

For more information on the `calwebb_ami3` pipeline stage visit the links below. 

> Stage description: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_ami3.html
>
> Pipeline code: https://github.com/spacetelescope/jwst/tree/master/jwst/ami

[Top of Page](#title_ID)

<a id="algorithm"></a>
# JWST CalWG Algorithm

`calwebb_ami3` is based on the `implaneia` algorithm:
> https://github.com/anand0xff/ImPlaneIA/tree/delivery

[Top of Page](#title_ID)

<a id="terms"></a>
# Defining Terms

Calibrator: reference star to measure PSF to calibrate out instrumental contributions to the interferometric observables 

PSF: point spread function

Target: source of interest for science program 


[Top of Page](#title_ID)

<a id="description"></a>
# Test Description

This test checks that simulated data runs through the `calwebb_detector1`, `calwebb_image2`, and `calwebb_ami3` steps of the pipeline without crashing. Association files are created for the target/calibrator pair at different dither positions. The notebook verifies that the `calwebb_ami3` runs on these association files.

[Top of Page](#title_ID)

<a id="data_descr"></a>
# Data Description

The data for this test are simulated AMI datasets that do not have bad pixels. The simulated source data is AB Dor, which is simulated with a 4-point dither pattern:

| Source         | Filename| Dither Position |
|:----------------|:---------|:-----------------|
|AB Dor (Target) |jw01093001001_01101_00005_nis_uncal.fits| 1|
| |jw01093001001_01101_00006_nis_uncal.fits| 2 |
| |jw01093001001_01101_00007_nis_uncal.fits| 3 |
| |jw01093001001_01101_00005_nis_uncal.fits| 4 |

HD 37093 is the PSF reference star, which is also simulated with a 4-point dither pattern.

| Source         | Filename| Dither Position |
|:----------------|:---------|:-----------------|
|HD 37093 (Calibrator)| jw01093002001_01101_00005_nis_uncal.fits | 1 |
| |jw01093002001_01101_00006_nis_uncal.fits | 2 |
| |jw01093002001_01101_00007_nis_uncal.fits | 3 |
| |jw01093002001_01101_00005_nis_uncal.fits | 4 |

Configuration files are also needed for the various `calwebb_ami3` steps:
- ami_analyze.cfg
- ami_normalize.cfg
- ami_average.cfg
- calwebb_ami3.cfig

Specific reference files are needed for the analysis, which also do not have bad pixels, and are loaded with this notebook.

[Top of Page](#title_ID)

<a id="tempdir"></a>
# Set up Temporary Directory


[Top of Page](#title_ID)

In [1]:
use_tempdir = True

# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
import shutil

if use_tempdir:
    data_dir = TemporaryDirectory()

    # Save original directory
    orig_dir = os.getcwd()

    # Move to new directory
    odir = data_dir.name
    os.chdir(data_dir.name)

# For info, print out where the script is running
print("Running in {}".format(os.getcwd()))

Running in /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p


In [2]:
# Choose CRDS cache location
use_local_crds_cache = False
crds_cache_tempdir = False
crds_cache_notebook_dir = True
crds_cache_home = False
crds_cache_custom_dir = False
crds_cache_dir_name = ""

if use_local_crds_cache:
    if crds_cache_tempdir:
        os.environ['CRDS_PATH'] = os.path.join(os.getcwd(), "crds")
    elif crds_cache_notebook_dir:
        try:
            os.environ['CRDS_PATH'] = os.path.join(orig_dir, "crds")
        except Exception as e:
            os.environ['CRDS_PATH'] = os.path.join(os.getcwd(), "crds")
    elif crds_cache_home:
        os.environ['CRDS_PATH'] = os.path.join(os.environ['HOME'], 'crds', 'cache')
    elif crds_cache_custom_dir:
        os.environ['CRDS_PATH'] = crds_cache_dir_name

<a id="imports"></a>
# Imports
List the package imports and why they are relevant to this notebook.

* astropy.io for opening fits files
* numpy for working with arrays
* IPython.display for printing markdown output
* jwst.datamodels for building model for JWST Pipeline
* jwst.pipeline.collect_pipeline_cfgs for gathering configuration files
* jwst.pipeline for initiating various pipeline stages
* jwst.ami to call the AMI Analyze step
* jwst.associations for using association files
* from ci_watson.artifactory_helpers import get_bigdata for reading data from Artifactory


[Top of Page](#title_ID)

In [3]:
from astropy.io import fits
import numpy as np
from IPython.display import Markdown
from jwst.datamodels import ImageModel
import jwst.datamodels as datamodels

from jwst.pipeline.collect_pipeline_cfgs import collect_pipeline_cfgs
from jwst.pipeline import Detector1Pipeline, Image2Pipeline, Image3Pipeline, Ami3Pipeline 
from jwst.ami import AmiAnalyzeStep  
from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base

from ci_watson.artifactory_helpers import get_bigdata

<a id="data_load"></a>
# Loading the Data
[Top of Page](#title_ID)

In [4]:
# Data files that will be imported by Artifactory:
datafiles = np.array(['jw01093001001_01101_00005_nis_uncal.fits',
                      'jw01093001001_01101_00006_nis_uncal.fits',
                      'jw01093001001_01101_00007_nis_uncal.fits',
                      'jw01093001001_01101_00008_nis_uncal.fits',
                      'jw01093002001_01101_00005_nis_uncal.fits',
                      'jw01093002001_01101_00006_nis_uncal.fits',
                      'jw01093002001_01101_00007_nis_uncal.fits',
                      'jw01093002001_01101_00008_nis_uncal.fits'])

# Read in reference files needed for this analysis (these don't have bad pixels, like simulations)
superbiasfile = get_bigdata('jwst_validation_notebooks',
                            'validation_data',
                            'ami_analyze',
                            'jwst_niriss_superbias_sim.fits')
darkfile = get_bigdata('jwst_validation_notebooks',
                       'validation_data',
                       'ami_analyze',
                       'jwst_niriss_dark_sub80_sim.fits')
flatfile = get_bigdata('jwst_validation_notebooks',
                       'validation_data',
                       'ami_analyze', 
                       'jwst_niriss_flat_general.fits')

# Read in configuration files
ami_analyze_cfg = get_bigdata('jwst_validation_notebooks',
                              'validation_data',
                              'ami_analyze',
                              'ami_analyze.cfg')
ami_normalize_cfg = get_bigdata('jwst_validation_notebooks',
                                'validation_data',
                                'ami_analyze',
                                'ami_normalize.cfg')
ami_average_cfg = get_bigdata('jwst_validation_notebooks',
                              'validation_data',
                              'ami_analyze',
                              'ami_average.cfg')
calwebb_ami3_cfg = get_bigdata('jwst_validation_notebooks',
                                'validation_data',
                                'ami_analyze',
                                'calwebb_ami3.cfg')

<a id="pipeline"></a>
# Run the Pipeline

Since this notebook tests whether the pipeline runs on all the datasets, we will run each stage of the pipeline in separate cells. That way, if a step fails, it will be easier to track down at what stage and step this error occurred.

[Top of Page](#title_ID)

## Run Detector1 stage of the pipeline to calibrate \*\_uncal.fits file

In [5]:
for file in datafiles:

    df = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'ami_analyze',
                     file)
    
    # Modify a keyword in each data file: only necessary for now
    # Next three lines are temporary to accommodate recent changes to Mirage and pipeline
    # and for Mirage to work with the pipeline.
    with datamodels.open(df) as model:
        model.meta.dither.dither_points = int(model.meta.dither.dither_points)
        model.save(df)

    # Run Detector1 stage of pipeline, specifying superbias and dark reference files
    result1 = Detector1Pipeline()
    result1.superbias.override_superbias = superbiasfile
    result1.dark_current.override_dark = darkfile
    result1.ipc.skip = True
    result1.save_results = True
    result1.output_dir = odir
    result1.run(df)

2021-10-04 15:53:01,058 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.


2021-10-04 15:53:01,060 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.


2021-10-04 15:53:01,063 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.


2021-10-04 15:53:01,064 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.


2021-10-04 15:53:01,068 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.


...

2021-10-04 15:59:12,813 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale done


2021-10-04 15:59:12,877 - stpipe.Detector1Pipeline - INFO - Saved model in /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/jw01093002001_01101_00008_nis_rateints.fits


2021-10-04 15:59:12,878 - stpipe.Detector1Pipeline - INFO - ... ending calwebb_detector1


2021-10-04 15:59:12,927 - stpipe.Detector1Pipeline - INFO - Saved model in /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/jw01093002001_01101_00008_nis_rate.fits


2021-10-04 15:59:12,927 - stpipe.Detector1Pipeline - INFO - Step Detector1Pipeline done


## Run Image2 stage of the pipeline to calibrate \*\_rate.fits file

In [6]:
for df in datafiles:

    # Run Image2 stage of the pipeline on the file created above to create rate file,
    # specifying flat field file
    df_rate = os.path.join(odir, os.path.basename(df.replace('uncal','rate')))
    result2 = Image2Pipeline()        
    result2.flat_field.override_flat = flatfile
    result2.photom.skip = True
    result2.resample.skip = True
    result2.save_results = True
    result2.output_dir = odir
    result2.run(df_rate)

2021-10-04 15:59:12,941 - stpipe.Image2Pipeline - INFO - Image2Pipeline instance created.


2021-10-04 15:59:12,943 - stpipe.Image2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.


2021-10-04 15:59:12,945 - stpipe.Image2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.


2021-10-04 15:59:12,946 - stpipe.Image2Pipeline.flat_field - INFO - FlatFieldStep instance created.


2021-10-04 15:59:12,947 - stpipe.Image2Pipeline.photom - INFO - PhotomStep instance created.


...

2021-10-04 15:59:23,476 - stpipe.Image2Pipeline.photom - INFO - Step photom done


2021-10-04 15:59:23,477 - stpipe.Image2Pipeline - INFO - Finished processing product /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/jw01093002001_01101_00008_nis


2021-10-04 15:59:23,478 - stpipe.Image2Pipeline - INFO - ... ending calwebb_image2


2021-10-04 15:59:23,572 - stpipe.Image2Pipeline - INFO - Saved model in /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/jw01093002001_01101_00008_nis_cal.fits


2021-10-04 15:59:23,573 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline done


## Run Image2 stage of the pipeline to calibrate \*\_rateints.fits file

In [7]:
for df in datafiles:

    # Run Image stage of the pipeline to create rateints file, specifying flat field file
    df_rateints = os.path.join(odir,os.path.basename(df.replace('uncal','rateints')))
    result3 = Image2Pipeline()
    result3.flat_field.override_flat = flatfile
    result3.photom.skip = True
    result3.resample.skip = True
    result3.save_results = True
    result3.output_dir = odir
    result3.run(df_rateints)    

2021-10-04 15:59:23,586 - stpipe.Image2Pipeline - INFO - Image2Pipeline instance created.


2021-10-04 15:59:23,588 - stpipe.Image2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.


2021-10-04 15:59:23,589 - stpipe.Image2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.


2021-10-04 15:59:23,591 - stpipe.Image2Pipeline.flat_field - INFO - FlatFieldStep instance created.


2021-10-04 15:59:23,592 - stpipe.Image2Pipeline.photom - INFO - PhotomStep instance created.


...

2021-10-04 15:59:34,861 - stpipe.Image2Pipeline.photom - INFO - Step photom done


2021-10-04 15:59:34,862 - stpipe.Image2Pipeline - INFO - Finished processing product /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/jw01093002001_01101_00008_nis


2021-10-04 15:59:34,863 - stpipe.Image2Pipeline - INFO - ... ending calwebb_image2


2021-10-04 15:59:34,972 - stpipe.Image2Pipeline - INFO - Saved model in /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/jw01093002001_01101_00008_nis_calints.fits


2021-10-04 15:59:34,973 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline done


## Run AmiAnalyze step on the \*\_cal.fits files created above

In [8]:
for df in datafiles:

    # Set up name of calibrated file
    df_cal = os.path.join(odir,os.path.basename(df.replace('uncal','cal')))
    
    # Run AMI Analyze Step of the pipeline
    result5 = AmiAnalyzeStep.call(df_cal, config_file = ami_analyze_cfg,
                                  output_dir = odir, save_results = True)

2021-10-04 15:59:35,054 - stpipe - WARNING - /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/stpipe/config_parser.py:118: ResourceWarning: unclosed file <_io.BufferedReader name='/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/ami_analyze.cfg'>
  return ConfigObj(config_file, raise_errors=True)



2021-10-04 15:59:35,056 - stpipe.ami_analyze - INFO - AmiAnalyzeStep instance created.


2021-10-04 15:59:35,232 - stpipe - WARNING - /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/stpipe/step.py:359: ResourceWarning: unclosed file <_io.FileIO name='/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/jw01093001001_01101_00005_nis_cal.fits' mode='rb' closefd=True>
  gc.collect()



2021-10-04 15:59:35,237 - stpipe.ami_analyze - INFO - Step ami_analyze running with args ('/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/jw01093001001_01101_00005_nis_cal.fits',).


2021-10-04 15:59:35,239 - stpipe.ami_analyze - INFO - Step ami_analyze parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': '/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p', 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'oversample': 3, 'rotation': 0.0, 'psf_offset': '0.0 0.0', 'rotation_search': '-3 3.1 1.'}


...

2021-10-04 16:01:04,057 - stpipe.ami_analyze - INFO - Initial values to use for psf offset = [0.0, 0.0]


2021-10-04 16:01:04,142 - stpipe.ami_analyze - INFO - Using filter throughput reference file /grp/crds/cache/references/jwst/jwst_niriss_throughput_0008.fits


2021-10-04 16:01:04,201 - stpipe.ami_analyze - INFO - Initial values to use for rotation search [-3. -2. -1.  0.  1.  2.  3.]


2021-10-04 16:01:16,091 - stpipe.ami_analyze - INFO - Saved model in /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/jw01093002001_01101_00008_nis_ami_analyze.fits


2021-10-04 16:01:16,092 - stpipe.ami_analyze - INFO - Step ami_analyze done


## Run AmiAnalyze on various target/calibrator pairs

Create association files to test calibration of target at different dither positions. Run AmiAnalyze on these association files. 

Note: the `program` and `targ_name` fields in the association files are the same for all pairs, so I have them set as default values in the `create_asn` routine.

Routine for creating association files (in \*.json format)

In [9]:
def create_asn(outdir, targ1, psf1, prod_name, out_file, asn_id, 
               program="1093_2_targets_f480m_2022.25coords_pipetesting", 
               targ_name='t001',
               targ2=None, psf2=None):
    
    # Create association file:
    asn = asn_from_list.asn_from_list([os.path.join(outdir,targ1)],
                                       product_name = prod_name,
                                       output_file = os.path.join(outdir,out_file),
                                       output_dir = outdir,rule = DMS_Level3_Base)
    
    asn['products'][0]['members'].append({'expname': os.path.join(odir,psf1),
                                          'exptype': 'psf'})
    
    # check whether 2nd set of target/calibrator pairs was inputted
    if targ2 is not None:   
        asn['products'][0]['members'].append({'expname':os.path.join(odir,targ2), 
                                              'exptype': 'science'})
        asn['products'][0]['members'].append({'expname':os.path.join(odir,psf2), 
                                              'exptype': 'psf'})



    asn['asn_type'] = 'ami3'
    asn['asn_id'] = asn_id
    asn['program'] = program
    asn['target'] = targ_name
    
    with open(os.path.join(outdir,out_file), 'w') as fp:
        fp.write(asn.dump()[1])
        fp.close()

### Create association files and run AmiAnalyze on these pairs

Association file 1 to calibrate average of targets at dithers 2 and 3 with the average of calibrators at dithers 2 and 3. 

In [10]:
asn1_file = "ami_asn001_targets23_cals23.json"

targ1 = "jw01093001001_01101_00006_nis_cal.fits"
psf1 = "jw01093002001_01101_00006_nis_cal.fits"
prod_name = "jw01093001001_01101"
asn_id = '001'

# Add second target/calibrator pair at this dither step
targ2 = "jw01093001001_01101_00007_nis_cal.fits"
psf2 = "jw01093002001_01101_00007_nis_cal.fits"

create_asn(odir, targ1, psf1, prod_name, asn1_file, asn_id, targ2=targ2, psf2=psf2)

# Run AmiAnalyze
Ami3Pipeline.call(asn1_file,config_file = calwebb_ami3_cfg,output_dir = odir)    

2021-10-04 16:01:16,464 - stpipe - WARNING - /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/stpipe/config_parser.py:118: ResourceWarning: unclosed file <_io.BufferedReader name='/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/calwebb_ami3.cfg'>
  return ConfigObj(config_file, raise_errors=True)



2021-10-04 16:01:16,467 - stpipe.Ami3Pipeline - INFO - Ami3Pipeline instance created.


2021-10-04 16:01:16,469 - stpipe.Ami3Pipeline.ami_analyze - INFO - AmiAnalyzeStep instance created.


2021-10-04 16:01:16,471 - stpipe.Ami3Pipeline.ami_average - INFO - AmiAverageStep instance created.


2021-10-04 16:01:16,472 - stpipe.Ami3Pipeline.ami_normalize - INFO - AmiNormalizeStep instance created.


...

2021-10-04 16:02:05,236 - stpipe.Ami3Pipeline.ami_normalize - INFO - Step ami_normalize done


2021-10-04 16:02:05,238 - stpipe.Ami3Pipeline - INFO - Blending metadata for PSF normalized target


2021-10-04 16:02:06,003 - stpipe.Ami3Pipeline - INFO - Saved model in /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/jw01093001001_01101_aminorm.fits


2021-10-04 16:02:06,005 - stpipe.Ami3Pipeline - INFO - ... ending calwebb_ami3


2021-10-04 16:02:06,006 - stpipe.Ami3Pipeline - INFO - Step Ami3Pipeline done


Association file 2 to calibrate target at POS1 with calibrator at POS1

In [11]:
# Create association file:
asn2_file = "ami_asn002_calibrate_targ1_cal1.json"

targ1 = "jw01093001001_01101_00005_nis_cal.fits"
psf1 = 'jw01093002001_01101_00005_nis_cal.fits'
prod_name = "jw01093001001_01101_00005cal00005"
asn_id = '002'

create_asn(odir, targ1, psf1,prod_name, asn2_file, asn_id)

# Run AmiAnalyze
Ami3Pipeline.call(asn2_file,config_file = calwebb_ami3_cfg,output_dir = odir)    

2021-10-04 16:02:06,367 - stpipe - WARNING - /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/stpipe/config_parser.py:118: ResourceWarning: unclosed file <_io.BufferedReader name='/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/calwebb_ami3.cfg'>
  return ConfigObj(config_file, raise_errors=True)



2021-10-04 16:02:06,371 - stpipe.Ami3Pipeline - INFO - Ami3Pipeline instance created.


2021-10-04 16:02:06,372 - stpipe.Ami3Pipeline.ami_analyze - INFO - AmiAnalyzeStep instance created.


2021-10-04 16:02:06,374 - stpipe.Ami3Pipeline.ami_average - INFO - AmiAverageStep instance created.


2021-10-04 16:02:06,375 - stpipe.Ami3Pipeline.ami_normalize - INFO - AmiNormalizeStep instance created.


...

2021-10-04 16:02:37,206 - stpipe.Ami3Pipeline.ami_normalize - INFO - Step ami_normalize done


2021-10-04 16:02:37,208 - stpipe.Ami3Pipeline - INFO - Blending metadata for PSF normalized target


2021-10-04 16:02:37,950 - stpipe.Ami3Pipeline - INFO - Saved model in /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/jw01093001001_01101_00005cal00005_aminorm.fits


2021-10-04 16:02:37,952 - stpipe.Ami3Pipeline - INFO - ... ending calwebb_ami3


2021-10-04 16:02:37,952 - stpipe.Ami3Pipeline - INFO - Step Ami3Pipeline done


Association file 3 to calibrate target at POS2 with calibrator at POS2

In [12]:
# Create association file:
asn3_file = "ami_asn003_calibrate_targ2_cal2.json"

targ1 = "jw01093001001_01101_00006_nis_cal.fits"
psf1 = "jw01093002001_01101_00006_nis_cal.fits"
prod_name = "jw01093001001_01101_00006cal00006"
asn_id = '003'

create_asn(odir, targ1, psf1, prod_name, asn3_file, asn_id)

# Run AmiAnalyze
Ami3Pipeline.call(asn3_file,config_file = calwebb_ami3_cfg,output_dir = odir)    

2021-10-04 16:02:38,297 - stpipe - WARNING - /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/stpipe/config_parser.py:118: ResourceWarning: unclosed file <_io.BufferedReader name='/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/calwebb_ami3.cfg'>
  return ConfigObj(config_file, raise_errors=True)



2021-10-04 16:02:38,300 - stpipe.Ami3Pipeline - INFO - Ami3Pipeline instance created.


2021-10-04 16:02:38,302 - stpipe.Ami3Pipeline.ami_analyze - INFO - AmiAnalyzeStep instance created.


2021-10-04 16:02:38,303 - stpipe.Ami3Pipeline.ami_average - INFO - AmiAverageStep instance created.


2021-10-04 16:02:38,305 - stpipe.Ami3Pipeline.ami_normalize - INFO - AmiNormalizeStep instance created.


...

2021-10-04 16:03:03,284 - stpipe.Ami3Pipeline.ami_normalize - INFO - Step ami_normalize done


2021-10-04 16:03:03,286 - stpipe.Ami3Pipeline - INFO - Blending metadata for PSF normalized target


2021-10-04 16:03:04,025 - stpipe.Ami3Pipeline - INFO - Saved model in /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/jw01093001001_01101_00006cal00006_aminorm.fits


2021-10-04 16:03:04,026 - stpipe.Ami3Pipeline - INFO - ... ending calwebb_ami3


2021-10-04 16:03:04,027 - stpipe.Ami3Pipeline - INFO - Step Ami3Pipeline done


Association file 4 to calibrate target at POS3 with calibrator at POS3

In [13]:
# Create association file:
asn4_file = "ami_asn004_calibrate_targ3_cal3.json"

targ1 = "jw01093001001_01101_00007_nis_cal.fits"
psf1 = "jw01093002001_01101_00007_nis_cal.fits"
prod_name = "jw01093001001_01101_00007cal00007"
asn_id = '004'

create_asn(odir, targ1, psf1, prod_name, asn4_file, asn_id)

# Run AmiAnalyze
Ami3Pipeline.call(asn3_file,config_file = calwebb_ami3_cfg,output_dir = odir)    

2021-10-04 16:03:04,370 - stpipe - WARNING - /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/stpipe/config_parser.py:118: ResourceWarning: unclosed file <_io.BufferedReader name='/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/calwebb_ami3.cfg'>
  return ConfigObj(config_file, raise_errors=True)



2021-10-04 16:03:04,373 - stpipe.Ami3Pipeline - INFO - Ami3Pipeline instance created.


2021-10-04 16:03:04,375 - stpipe.Ami3Pipeline.ami_analyze - INFO - AmiAnalyzeStep instance created.


2021-10-04 16:03:04,376 - stpipe.Ami3Pipeline.ami_average - INFO - AmiAverageStep instance created.


2021-10-04 16:03:04,377 - stpipe.Ami3Pipeline.ami_normalize - INFO - AmiNormalizeStep instance created.


...

2021-10-04 16:03:29,866 - stpipe.Ami3Pipeline.ami_normalize - INFO - Step ami_normalize done


2021-10-04 16:03:29,868 - stpipe.Ami3Pipeline - INFO - Blending metadata for PSF normalized target


2021-10-04 16:03:30,630 - stpipe.Ami3Pipeline - INFO - Saved model in /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/jw01093001001_01101_00006cal00006_aminorm.fits


2021-10-04 16:03:30,632 - stpipe.Ami3Pipeline - INFO - ... ending calwebb_ami3


2021-10-04 16:03:30,633 - stpipe.Ami3Pipeline - INFO - Step Ami3Pipeline done


Association file 5 to calibrate target at POS4 with calibrator at POS4

In [14]:
# Create association file:
asn5_file = "ami_asn005_calibrate_targ4_cal4.json"

targ1 = "jw01093001001_01101_00008_nis_cal.fits"
psf1 = "jw01093002001_01101_00008_nis_cal.fits"
prod_name = "jw01093001001_01101_00008cal00008"
asn_id = '005'

create_asn(odir, targ1, psf1, prod_name, asn5_file, asn_id)

# Run AmiAnalyze
Ami3Pipeline.call(asn3_file,config_file = calwebb_ami3_cfg,output_dir = odir)    

2021-10-04 16:03:30,986 - stpipe - WARNING - /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/stpipe/config_parser.py:118: ResourceWarning: unclosed file <_io.BufferedReader name='/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/calwebb_ami3.cfg'>
  return ConfigObj(config_file, raise_errors=True)



2021-10-04 16:03:30,991 - stpipe.Ami3Pipeline - INFO - Ami3Pipeline instance created.


2021-10-04 16:03:30,993 - stpipe.Ami3Pipeline.ami_analyze - INFO - AmiAnalyzeStep instance created.


2021-10-04 16:03:30,995 - stpipe.Ami3Pipeline.ami_average - INFO - AmiAverageStep instance created.


2021-10-04 16:03:30,996 - stpipe.Ami3Pipeline.ami_normalize - INFO - AmiNormalizeStep instance created.


...

2021-10-04 16:03:55,984 - stpipe.Ami3Pipeline.ami_normalize - INFO - Step ami_normalize done


2021-10-04 16:03:55,986 - stpipe.Ami3Pipeline - INFO - Blending metadata for PSF normalized target


2021-10-04 16:03:56,734 - stpipe.Ami3Pipeline - INFO - Saved model in /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/jw01093001001_01101_00006cal00006_aminorm.fits


2021-10-04 16:03:56,735 - stpipe.Ami3Pipeline - INFO - ... ending calwebb_ami3


2021-10-04 16:03:56,736 - stpipe.Ami3Pipeline - INFO - Step Ami3Pipeline done


Association file 6 to calibrate calibrator at POS2 with calibrator at POS3

In [15]:
# Create association file:
asn6_file = "ami_asn006_calibrate_cal2_cal3.json"

targ1 = "jw01093002001_01101_00006_nis_cal.fits"
psf1 = "jw01093002001_01101_00007_nis_cal.fits"
prod_name = "jw01093002001_01101_00006cal00007"
asn_id = '006'

create_asn(odir, targ1, psf1, prod_name, asn6_file, asn_id, targ_name='t002')

# Run AmiAnalyze
Ami3Pipeline.call(asn3_file,config_file = calwebb_ami3_cfg,output_dir = odir)    

2021-10-04 16:03:57,084 - stpipe - WARNING - /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/stpipe/config_parser.py:118: ResourceWarning: unclosed file <_io.BufferedReader name='/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/calwebb_ami3.cfg'>
  return ConfigObj(config_file, raise_errors=True)



2021-10-04 16:03:57,087 - stpipe.Ami3Pipeline - INFO - Ami3Pipeline instance created.


2021-10-04 16:03:57,089 - stpipe.Ami3Pipeline.ami_analyze - INFO - AmiAnalyzeStep instance created.


2021-10-04 16:03:57,090 - stpipe.Ami3Pipeline.ami_average - INFO - AmiAverageStep instance created.


2021-10-04 16:03:57,092 - stpipe.Ami3Pipeline.ami_normalize - INFO - AmiNormalizeStep instance created.


...

2021-10-04 16:04:22,158 - stpipe.Ami3Pipeline.ami_normalize - INFO - Step ami_normalize done


2021-10-04 16:04:22,159 - stpipe.Ami3Pipeline - INFO - Blending metadata for PSF normalized target


2021-10-04 16:04:22,921 - stpipe.Ami3Pipeline - INFO - Saved model in /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/jw01093001001_01101_00006cal00006_aminorm.fits


2021-10-04 16:04:22,923 - stpipe.Ami3Pipeline - INFO - ... ending calwebb_ami3


2021-10-04 16:04:22,924 - stpipe.Ami3Pipeline - INFO - Step Ami3Pipeline done


Association file 7 to calibrate calibrator at POS3 with calibrator at POS2

In [16]:
# Create association file:
asn7_file = "ami_asn007_calibrate_cal3_cal2.json"

targ1 = "jw01093002001_01101_00007_nis_cal.fits"
psf1 = "jw01093002001_01101_00006_nis_cal.fits"
prod_name = "jw01093002001_01101_00007cal00006"
asn_id = '007'

create_asn(odir, targ1, psf1, prod_name, asn7_file, asn_id, targ_name='t002')

# Run AmiAnalyze
Ami3Pipeline.call(asn3_file,config_file = calwebb_ami3_cfg,output_dir = odir)    

2021-10-04 16:04:23,267 - stpipe - WARNING - /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/stpipe/config_parser.py:118: ResourceWarning: unclosed file <_io.BufferedReader name='/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/calwebb_ami3.cfg'>
  return ConfigObj(config_file, raise_errors=True)



2021-10-04 16:04:23,270 - stpipe.Ami3Pipeline - INFO - Ami3Pipeline instance created.


2021-10-04 16:04:23,272 - stpipe.Ami3Pipeline.ami_analyze - INFO - AmiAnalyzeStep instance created.


2021-10-04 16:04:23,273 - stpipe.Ami3Pipeline.ami_average - INFO - AmiAverageStep instance created.


2021-10-04 16:04:23,274 - stpipe.Ami3Pipeline.ami_normalize - INFO - AmiNormalizeStep instance created.


...

2021-10-04 16:04:48,261 - stpipe.Ami3Pipeline.ami_normalize - INFO - Step ami_normalize done


2021-10-04 16:04:48,262 - stpipe.Ami3Pipeline - INFO - Blending metadata for PSF normalized target


2021-10-04 16:04:49,006 - stpipe.Ami3Pipeline - INFO - Saved model in /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpbj6nhh1p/jw01093001001_01101_00006cal00006_aminorm.fits


2021-10-04 16:04:49,007 - stpipe.Ami3Pipeline - INFO - ... ending calwebb_ami3


2021-10-04 16:04:49,008 - stpipe.Ami3Pipeline - INFO - Step Ami3Pipeline done


<a id="testing"></a>
# Test Results

Did the above cells run without errors? If so, **huzzah** the test passed! 

If not, track down why the pipeline failed to run on these datasets.

[Top of Page](#title_ID)

<a id="about_ID"></a>
## About this Notebook
**Authors:** Deepashri Thatte, Senior Staff Scientist, NIRISS
<br>Stephanie LaMassa, Scientist, NIRISS
<br>**Updated On:** 08/04/2021

[Top of Page](#title_ID)
<img style="float: right;" src="./stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="stsci_pri_combo_mark_horizonal_white_bkgd" width="200px"/> 